# Test notebook for Acceptance Test Campaign related to LSST Science Pipelines Release 20.0

This test will be executed on the LSST Science Platform Notebook Aspect, initialized with Science Pipelines release `r20-0-0`.

### Test case LVV-T28: Verify implementation of measurements in catalogs from PVIs
Verify that source measurements in catalogs are in flux units.

In [1]:
# Confirm that the version of the Science Pipelines is v20_0_0:
! echo $HOSTNAME
! eups list -s | grep lsst_distrib

nb-jcarlin-r20-0-0
lsst_distrib          20.0.0     	current v20_0_0 setup


In [2]:
import lsst.daf.persistence as dafPersist

In [3]:
import numpy as np

We will use HSC-RC2, as processed using `v20_0_0_rc1`, which is the pipelines version that was used to create `v20_0_0`.

In [4]:
# The output repo is tagged with the Jira ticket number "DM-25349":
rc2_repo = '/datasets/hsc/repo/rerun/RC/v20_0_0_rc1/DM-25349'

In [5]:
# Initialize the butler repo:
butler = dafPersist.Butler(rc2_repo)

### Check a visit image:

In [6]:
# Select an arbitrary source catalog from a calexp (PVI):
filter = 'HSC-I'
# PVI:
ccd = 14
visit = 35890

In [7]:
dataIdPVI = {'filter':filter, 'visit':visit, 'ccd':ccd}
src = butler.get('src', dataId = dataIdPVI)

### Check that the schema for this source catalog contains Flux measurements (and that they are in units of "counts")

In [8]:
sch = src.schema

#### Sources in processed visit images

In [9]:
# Check the units on each entry:
src_flag = []
for a in sch:
    nnn = a.getField().getName()
    if 'instFlux' in nnn and 'flag' not in nnn:
        flux_units = a.getField().getUnits()
        src_flag.append('count' in flux_units)
        print(f'{nnn:60}..... {flux_units:20}')

deblend_psf_instFlux                                        ..... count               
base_Blendedness_raw_child_instFlux                         ..... count               
base_Blendedness_raw_parent_instFlux                        ..... count               
base_Blendedness_abs_child_instFlux                         ..... count               
base_Blendedness_abs_parent_instFlux                        ..... count               
base_SdssShape_instFlux                                     ..... count               
base_SdssShape_instFluxErr                                  ..... count               
base_SdssShape_instFlux_xx_Cov                              ..... count*pixel^2       
base_SdssShape_instFlux_yy_Cov                              ..... count*pixel^2       
base_SdssShape_instFlux_xy_Cov                              ..... count*pixel^2       
base_CircularApertureFlux_3_0_instFlux                      ..... count               
base_CircularApertureFlux_3_0_instFluxErr  

In [12]:
assert(np.all(src_flag)), 'FALSE: not all instFlux entries have units of counts.'
print('All src instFlux entries have units of counts: ',np.all(src_flag))

All src instFlux entries have units of counts:  True
